# Preparation
## Paths

In [ ]:
# Import Operating System
import os

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
print(ipynb_path)

In [ ]:
root = "\\".join(ipynb_path.split("\\")[:-2])+"\\datasets"
print(root)

In [ ]:
workspace = "custom"

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join(root, workspace),
    'OVERLAYS_PATH': os.path.join(root, workspace,'overlays'),
    'BACKGROUNDS_PATH': os.path.join(root, workspace,'backgrounds'),
    'IMAGES_PATH': os.path.join(root, workspace,'images'),
    'ANNOTATIONS_PATH': os.path.join(root, workspace,'annotations'),
 }
print(paths)

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Backgrounds

In [ ]:
_, _, background_files = next(os.walk(paths['BACKGROUNDS_PATH']), (None, [], []))
print("background files: " + str(len(background_files)))

## Overlays

In [ ]:
_, overlays_folders, overlays_files = next(os.walk(paths['OVERLAYS_PATH']), (None, [], []))
print("overlays folders: " + str(len(overlays_folders)))
print("overlays files: " + str(len(overlays_files)))

## Labels

In [ ]:
labels = []
overlays = []
for folder in overlays_folders:
    #print("label: " + folder)
    labels.append(folder)
    _, _, overlays_files_versions = next(os.walk(os.path.join(paths['OVERLAYS_PATH'], folder)), (None, [], []))
    for file in overlays_files_versions:
        #print("file: " + file)
        overlays.append(os.path.join(paths['OVERLAYS_PATH'], folder, file))

for file in overlays_files:
    overlays.append(os.path.join(paths['OVERLAYS_PATH'], file))
    file_label = "-".join(file.split("-")[:-1])
    #print("label: " + file_label)
    labels.append(file_label)

#print(labels)
print(overlays)

list_filename = os.path.join(paths['WORKSPACE_PATH'], 'className.list')
#print(list_filename)

with open(list_filename, 'w') as file :
    file.write('')

for (l, label) in enumerate(labels):
    with open(list_filename, 'a') as file :
        file.write(label+"\n")

In [ ]:
#!pip install opencv-python
#!pip install matplotlib
#!pip install scipy
#!pip install imutils

In [ ]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import math
import random

In [ ]:
def crop_alpha(image_to_crop):
    # axis 0 is the row(y) and axis(x) 1 is the column
    # get the nonzero alpha coordinates
    y, x = image_to_crop[:, :, 3].nonzero()
    minx = np.min(x)
    miny = np.min(y)
    maxx = np.max(x)
    maxy = np.max(y)
    croped = image_to_crop[miny:maxy, minx:maxx]
    return croped

In [ ]:
def rotate_image(image_to_rotate, rmax=45, rmin=0):
    deg = random.uniform(rmin, rmax)
    pos = 2*(random.random()-0.5)
    
    height, width = image_to_rotate.shape[:2]
    dia = int(math.sqrt(math.pow(height, 2) + math.pow(width, 2)))

    n_channels = 4
    transparent_img = np.zeros((dia, dia, n_channels), dtype=np.single)

    # padding_y
    bpy = int((dia - height)/2)
    # padding_x
    bpx = int((dia - width)/2)

    transparent_img[bpy:(bpy+height), bpx:(bpx+width)] = image_to_rotate

    rotated = imutils.rotate(transparent_img, angle=deg*pos)
    return rotated

In [ ]:
def adjust_image(image_to_adjust, gmin=0.5, gmax=1.5, smin=0.5, smax=1.0):
    saturation = random.uniform(smin, smax)
    gamma = random.uniform(gmin, gmax)

    a = rotated[:, :, 3]

    hsv = cv2.cvtColor(image_to_adjust, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    s_desat = cv2.multiply(s, saturation)
    v_gamma = cv2.multiply(v, gamma)
    hsv_new = cv2.merge([h, s_desat, v_gamma])

    bgr_adjust = cv2.cvtColor(hsv_new, cv2.COLOR_HSV2BGR)
    adjusted = cv2.cvtColor(bgr_adjust, cv2.COLOR_RGB2RGBA)
    adjusted[:, :, 3] = a
    return adjusted

In [ ]:
def resize_image(image_to_resize, max=50, min=0):
    resize = random.uniform(min, max)
    procent = 1-resize/100

    height, width = image_to_resize.shape[:2]
    dim = (int(width*procent), height)
    resized = cv2.resize(image_to_resize, dim, interpolation=cv2.INTER_AREA)
    return resized

In [ ]:
img = mpimg.imread(overlays[0])
cropImg = crop_alpha(img)

rotated = rotate_image(cropImg)

adjusted = adjust_image(rotated)

resized = resize_image(adjusted)

resized*=255
resized = resized.astype(int)
resized[resized < 0] = 0
resized[resized > 255] = 255

imgplot = plt.imshow(crop_alpha(resized))
#print(cropImg_)